In [1]:
import gzip
import pickle
import numpy as np
import pandas as pd

from pathlib import Path
from matplotlib import pyplot

import torch
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

In [2]:
def _split_and_shuffle_labels(y_data, seed):
    num_of_class = len(set(y_data.tolist()))
    y_data=pd.DataFrame(y_data, columns=['label'])
    y_data['index'] = np.arange(len(y_data))
    label_dict = dict()
    cur_idx = list()

    for i in range(num_of_class):
        var_name = 'label' + str(i)
        label_info = y_data[y_data['label'] == i]
        np.random.seed(seed)
        label_info = np.random.permutation(label_info)
        label_info = pd.DataFrame(label_info, columns=['label', 'index'])
        label_dict.update({var_name: label_info })
        cur_idx.append(0)

    return label_dict, cur_idx

In [3]:
def _get_iid_subsamples_indices(y_data, number_of_samples, seed):
    num_of_class = len(set(y_data.tolist()))
    label_dict, cur_idx = _split_and_shuffle_labels(y_data, seed)
    sample_dict = dict()
    dist = 1.0 / num_of_class
    for i in range(number_of_samples):
        sample_name = 'sample' + str(i)
        dumb = pd.DataFrame()
        for j in range(num_of_class):
            label_name = str('label') + str(j)
            if i == (number_of_samples - 1):
                next_idx = len(label_dict[label_name])
            else:
                next_idx = int(len(label_dict[label_name]) * dist)
                next_idx += cur_idx[j]
            temp = label_dict[label_name][cur_idx[j]:next_idx]
            dumb=pd.concat([dumb, temp], axis=0)
            cur_idx[j] = next_idx
        dumb.reset_index(drop=True, inplace=True)    
        sample_dict.update({sample_name: dumb}) 
    return sample_dict

In [4]:
def _get_non_iid_subsamples_indices(y_data, number_of_samples, pdist, seed):    
    num_of_class = len(set(y_data.tolist()))
    label_dict, cur_idx = _split_and_shuffle_labels(y_data, seed)
    sample_dict = dict()
    for i in range(number_of_samples):
        sample_name = 'sample' + str(i)
        dumb = pd.DataFrame()
        dist1 = pdist * (2 / 3)
        dist2 = pdist - dist1
        dist3 = (1.0 - pdist) / (num_of_class - 2)
        for j in range(num_of_class):
            label_name = str('label') + str(j)
            dist = dist1 if j == i else dist2 if (j % 5) == (i % 5) else dist3
            if i == (number_of_samples - 1):
                next_idx = len(label_dict[label_name])
            else:
                next_idx = int(len(label_dict[label_name]) * dist)
                next_idx += cur_idx[j]
            temp = label_dict[label_name][cur_idx[j]:next_idx]
            dumb = pd.concat([dumb, temp], axis=0)
            cur_idx[j] = next_idx
        dumb.reset_index(drop=True, inplace=True)    
        sample_dict.update({sample_name: dumb}) 
    return sample_dict 

In [5]:
def _create_subsamples(sample_dict, x_data, y_data, x_name, y_name):
    x_data_dict= dict()
    y_data_dict= dict()
    
    for i in range(len(sample_dict)):  ### len(sample_dict)= number of samples
        xname= x_name+str(i)
        yname= y_name+str(i)
        sample_name="sample"+str(i)
        
        indices=np.sort(np.array(sample_dict[sample_name]['index']))
        
        x_info= x_data[indices,:]
        x_data_dict.update({xname : x_info})
        
        y_info= y_data[indices]
        y_data_dict.update({yname : y_info})
        
    return x_data_dict, y_data_dict

In [6]:
def _print_dict(x_train_dict, y_train_dict, x_test_dict, y_test_dict):
    sum = 0
    print('[*] Train Dataset (x, y)')
    for idx, (x_key, y_key) in enumerate(zip(x_train_dict, y_train_dict)):
        sum += len(x_train_dict[x_key])
        print('- sample{}: {}, {}'.format(idx, len(x_train_dict[x_key]), len(y_train_dict[y_key])))
        print(': ', end='')
        for i in range(10):
            print(y_train_dict[y_key].tolist().count(i), end=' ')
        print('')
    print('# total:', sum, end='\n\n')

    sum = 0
    print('[*] Test Dataset (x, y)')
    for idx, (x_key, y_key) in enumerate(zip(x_test_dict, y_test_dict)):
        sum += len(x_test_dict[x_key])
        print('- sample{}: {}, {}'.format(idx, len(x_test_dict[x_key]), len(y_test_dict[y_key])))
        print(': ', end='')
        for i in range(10):
            print(y_test_dict[y_key].tolist().count(i), end=' ')
        print('')
    print('# total:', sum)

In [7]:
def load_mnist_data(path='./data/mnist.pkl.gz', torch_tensor=True):
    data_path = Path(path)
    with gzip.open(data_path, "rb") as f:
        ((x_train, y_train), (x_test, y_test)) = pickle.load(f)
        
    if torch_tensor:
        x_train, y_train, x_test, y_test = map(torch.tensor, (x_train, y_train, x_test, y_test))
    print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)
    
    return x_train, y_train, x_test, y_test

In [8]:
def create_non_iid_samples(x_train, y_train, x_test, y_test, num_of_sample=10, pdist=0.6, seed=1, verbose=True):
    sample_dict_train = _get_non_iid_subsamples_indices(y_train, num_of_sample, pdist, seed)
    x_train_dict, y_train_dict = _create_subsamples(sample_dict_train, x_train, y_train, 'x_train', 'y_train')
    
    sample_dict_test = _get_non_iid_subsamples_indices(y_test, num_of_sample, pdist, seed)
    x_test_dict, y_test_dict = _create_subsamples(sample_dict_test, x_test, y_test, 'x_test', 'y_test')
    
    if verbose:
        _print_dict(x_train_dict, y_train_dict, x_test_dict, y_test_dict)
    return x_train_dict, y_train_dict, x_test_dict, y_test_dict

In [9]:
def create_iid_samples(x_train, y_train, x_test, y_test, num_of_sample=10, seed=1, verbose=True):
    sample_dict_train = _get_iid_subsamples_indices(y_train, num_of_sample, seed)
    x_train_dict, y_train_dict = _create_subsamples(sample_dict_train, x_train, y_train, 'x_train', 'y_train')
    
    sample_dict_test = _get_iid_subsamples_indices(y_test, num_of_sample, seed)
    x_test_dict, y_test_dict = _create_subsamples(sample_dict_test, x_test, y_test, 'x_test', 'y_test')
    
    if verbose:
        _print_dict(x_train_dict, y_train_dict, x_test_dict, y_test_dict)
    return x_train_dict, y_train_dict, x_test_dict, y_test_dict

In [10]:
def create_dataloader(x_train, y_train, x_test, y_test, batch_size):
    train_data = None
    test_data = None
    
    if x_train != None and y_train != None:
        train_data = DataLoader(TensorDataset(x_train, y_train), batch_size=batch_size, shuffle=True)
    if x_test != None and y_test != None:
        test_data = DataLoader(TensorDataset(x_test, y_test), batch_size=1)
    
    return train_data, test_data